In [1]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
print('finish import')

finish import


In [2]:
package_names = ('lme4', 'lmerTest', 'emmeans', "geepack", "afex", "dplyr", 'reticulate')

if all(rpackages.isinstalled(x) for x in package_names):
   have_package=True
else:
   have_package=False
       
if not have_package:
       utils = importr("utils")
       utils.chooseCRANmirror(ind=1)
       packages_to_install = [x for x in package_names if not rpackages.isinstalled(x)]
       print(packages_to_install)
       if len(packages_to_install)>0:
           utils.install_packages(StrVector(packages_to_install))

print("done")

done


In [3]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(lme4)
library(lmerTest)
data(dietox, package='geepack')
m<-lmer('Weight ~ Time + (1+Time|Pig)', data=dietox)
print(summary(m))

In [4]:
%%R
library(lme4)
library(afex)
library(emmeans)
library(dplyr)
library(reticulate)
fhch <- get(data("fhch2010")) %>% # load data from afex %>% 
  filter(correct == TRUE) %>% # subset the data to correct answers 
  select(id, task, stimulus, item, rt, log_rt) # keep only used cols

R[write to console]: Loading required package: Matrix

R[write to console]: ************
Welcome to afex. For support visit: http://afex.singmann.science/

R[write to console]: - Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'S', 'KR', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- NEWS: emmeans() for ANOVA models now uses model = 'multivariate' as default.
- Get and set global package options with: afex_options()
- Set orthogonal sum-to-zero contrasts globally: set_sum_contrasts()
- For example analyses see: browseVignettes("afex")
************

R[write to console]: 
Attaching package: ‘afex’


R[write to console]: The following object is masked from ‘package:lme4’:

    lmer


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked fro


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [5]:
%%R
head(fhch, 10)

   id   task stimulus   item    rt      log_rt
1  N1 naming     word potted 1.091  0.08709471
2  N1 naming     word engine 0.876 -0.13238919
3  N1 naming     word  ideal 0.710 -0.34249031
4  N1 naming  nonword  uares 1.210  0.19062036
5  N1 naming  nonword   xazz 0.843 -0.17078832
6  N1 naming     word   fill 0.785 -0.24207156
7  N1 naming  nonword bounge 0.662 -0.41248972
8  N1 naming  nonword  psems 0.713 -0.33827386
9  N1 naming     word   jolt 0.750 -0.28768207
10 N1 naming  nonword flepts 0.779 -0.24974423


In [6]:
%%R
m <- lme4::lmer(
  log_rt ~ 1 + task * stimulus + 
    (1 | id) +
    (1 | item), 
  data = fhch
)
summary(m)

Linear mixed model fit by REML ['lmerMod']
Formula: log_rt ~ 1 + task * stimulus + (1 | id) + (1 | item)
   Data: fhch

REML criterion at convergence: 7387.7

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.0260 -0.6083 -0.1062  0.4883  5.2923 

Random effects:
 Groups   Name        Variance Std.Dev.
 item     (Intercept) 0.003497 0.05914 
 id       (Intercept) 0.037428 0.19346 
 Residual             0.099088 0.31478 
Number of obs: 12960, groups:  item, 600; id, 45

Fixed effects:
                           Estimate Std. Error t value
(Intercept)                -0.35293    0.04378  -8.061
tasklexdec                  0.34360    0.05857   5.866
stimulusnonword             0.31202    0.00957  32.604
tasklexdec:stimulusnonword -0.25856    0.01118 -23.135

Correlation of Fixed Effects:
            (Intr) tsklxd stmlsn
tasklexdec  -0.743              
stimlsnnwrd -0.109  0.060       
tsklxdc:stm  0.069 -0.095 -0.638


In [8]:
%%R
emm <- emmeans(m, ~task*stimulus, lmer.df = "a")
emm

 task   stimulus   emmean     SE  df asymp.LCL asymp.UCL
 naming word     -0.35293 0.0438 Inf   -0.4387   -0.2671
 lexdec word     -0.00933 0.0392 Inf   -0.0862    0.0675
 naming nonword  -0.04091 0.0438 Inf   -0.1267    0.0449
 lexdec nonword   0.04413 0.0392 Inf   -0.0327    0.1210

Degrees-of-freedom method: asymptotic 
Confidence level used: 0.95 


In [10]:
%%R
pairs(emm, adjust = "BH")

 contrast                        estimate      SE  df z.ratio p.value
 naming word - lexdec word        -0.3436 0.05857 Inf  -5.866  <.0001
 naming word - naming nonword     -0.3120 0.00957 Inf -32.604  <.0001
 naming word - lexdec nonword     -0.3971 0.05877 Inf  -6.756  <.0001
 lexdec word - naming nonword      0.0316 0.05877 Inf   0.537  0.5911
 lexdec word - lexdec nonword     -0.0535 0.00894 Inf  -5.979  <.0001
 naming nonword - lexdec nonword  -0.0850 0.05858 Inf  -1.452  0.1759

Degrees-of-freedom method: asymptotic 
P value adjustment: BH method for 6 tests 
